In [ ]:
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_community.llms import Ollama

# Set up your local LLM
llm = Ollama(model="mistral")

# Define prompt templates
summary_prompt = PromptTemplate.from_template("Summarize this text:\n\n{text}")
sentiment_prompt = PromptTemplate.from_template("Classify the sentiment of this summary:\n\n{summary}")
tweet_prompt = PromptTemplate.from_template("Write a tweet based on this summary:\n\n{summary}")

# Create runnable chains
summary_chain = summary_prompt | llm | StrOutputParser()
sentiment_chain = sentiment_prompt | llm | StrOutputParser()
tweet_chain = tweet_prompt | llm | StrOutputParser()

# Compose full chain
chain = (
    summary_chain
    | (lambda summary: {"summary": summary})
    | {
        "summary": lambda x: x["summary"],
        "sentiment": sentiment_chain,
        "tweet": tweet_chain
    }
)

# Interactive loop
print("🧠 Tweet Generator with Sentiment Analysis (LangChain + Mistral)")
print("Type a paragraph to summarize, classify, and tweet. Type 'exit' to quit.\n")

while True:
    user_input = input("📝 Your input: ").strip()
    if user_input.lower() in ["exit", "quit"]:
        print("👋 Goodbye!")
        break
    if not user_input:
        print("⚠️ Please enter some text.")
        continue

    result = chain.invoke({"text": user_input})
    print("📝 Summary:", result["summary"])
    print("📊 Sentiment:", result["sentiment"])
    print("🐦 Tweet:", result["tweet"], "\n")

🧠 Tweet Generator with Sentiment Analysis (LangChain + Mistral)
Type a paragraph to summarize, classify, and tweet. Type 'exit' to quit.

📝 Summary:  The text is a review of a blender that the author finds impressive. They use it daily for protein shakes and commend its ability to quickly pulverize frozen fruit and ice without leaving chunks. Cleaning is also easy, making it a valuable addition. Overall, they believe it's worth every penny and strongly recommend it due to its significant impact on their breakfast routine.
📊 Sentiment:  The sentiment of this summary is overwhelmingly positive. The author finds the blender impressive, commends its functionality, ease of cleaning, and value for money, and strongly recommends it, indicating high satisfaction with the product.
🐦 Tweet:  "Revolutionizing my mornings ☀️ Just tried the blender that pulverizes frozen fruit & ice like a pro, leaving no chunks in my protein shakes 🍌🥦🚀 It's easy to clean and a game-changer for breakfast! Absolutel